In [1]:
import os
import re
import glob
import numpy as np
import pandas as pd
import sklearn.decomposition

In [2]:
### example snakemake: /hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/da_coloc_GxC/DA_colocs_GxC_ATG10_top_snakemake.py 

In [3]:
# as it is it tests all cis SNPs which is useful to plot a Manhattan, 
# but for box-plots just observing effect sizes not needed

In [4]:
mydir = "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/May2021/genetic_effect/MOFA10/flip_signs/"

In [37]:
myfiles = glob.glob(mydir+"*mean_donor_level.csv")
len(myfiles)

188

In [38]:
n_genes = 50000
n_pcs = 15
for file in myfiles:
    cat = re.sub("/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/May2021/genetic_effect/MOFA10/flip_signs/", "", 
             re.sub("_mean_donor_level.csv","",file))
    outdir = "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/May2021/genetic_effect/MOFA10/flip_signs/input_files_"+cat
    phenotype_file = os.path.join(outdir, 'phenotypes.tsv')
    covariates_file = os.path.join(outdir, 'covariates.tsv')
    noise_matrix_file = os.path.join(outdir, 'noise_matrix.tsv')
    if not os.path.exists(outdir):
        os.makedirs(outdir, exist_ok=True)
    phenotype_df = pd.read_csv(file, index_col=0)
    cell_count_ds = phenotype_df['n_cells']
    # noise matrix (1/n)
    noise_scaling_vector = [1/float(x) for x in cell_count_ds.tolist()]
    noise_matrix = np.diag(noise_scaling_vector)
    noise_matrix_df = pd.DataFrame(data=noise_matrix, index=cell_count_ds.index, columns=cell_count_ds.index)
    noise_matrix_df.to_csv(noise_matrix_file, sep='\t')
    # phenotype (pseudobulk, mean)
    #load map HGNC to ensembl
    mapping_df = pd.read_csv('/nfs/leia/research/stegle/dseaton/genomes/hg19/annotation/geneid_mappings/hgnc_symbol2ensembl_gene_id.txt', sep='\t')
    cols_to_drop = list(set(phenotype_df.columns) & {'n_cells'})
    phenotype_df = phenotype_df.drop(cols_to_drop, axis=1)
    phenotype_df = phenotype_df.transpose()
    gene_names = phenotype_df.index
    selected_genes = list(phenotype_df.mean(axis=1).nlargest(n_genes).index)
    phenotype_df = phenotype_df.loc[selected_genes, :]
    phenotype_list = phenotype_df.index
    mapping_df = mapping_df.query('hgnc_symbol in @phenotype_list')
    mapping_df = mapping_df.drop_duplicates(subset=['hgnc_symbol'])
    mapping_df = mapping_df.set_index('hgnc_symbol')
    phenotype_df = phenotype_df.loc[mapping_df.index,:]
    phenotype_df.index = mapping_df['ensembl_gene_id']
    phenotype_df.to_csv(phenotype_file, sep='\t')
    # covariates (15 expression PCs)
    pc_mat = sklearn.decomposition.PCA(n_components=n_pcs).fit_transform(phenotype_df.values.transpose())
    pc_df = pd.DataFrame(data=pc_mat, index=phenotype_df.columns, columns=['PC{}'.format(x) for x in range(1,n_pcs+1)])
    pc_df.to_csv(covariates_file, sep='\t')

In [15]:
#load map HGNC to ensembl
mapping_df = pd.read_csv('/nfs/leia/research/stegle/dseaton/genomes/hg19/annotation/geneid_mappings/hgnc_symbol2ensembl_gene_id.txt', sep='\t')

In [16]:
phenotype_df = pd.read_csv(file, index_col=0)

In [17]:
phenotype_df.head()

,MIR1302-10,FAM138A,OR4F5,RP11-34P13.7,RP11-34P13.8,AL627309.1,RP11-34P13.14,RP11-34P13.9,AP006222.2,RP4-669L17.10,...,AL590523.1,CT476828.1,PNRC2-1,SRSF10-1,AC145205.1,BAGE5,CU459201.1,AC002321.2,AC002321.1,n_cells
HPSI0114i-bezi_3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.309524,0.000000,...,0.0,0.0,0.071429,0.142857,0.0,0.0,0.0,0.0,0.0,42
HPSI0114i-eipl_1,0.0,0.0,0.0,0.0,0.0,0.031008,0.0,0.0,0.472868,0.000000,...,0.0,0.0,0.000000,0.131783,0.0,0.0,0.0,0.0,0.0,129
HPSI0114i-kolf_2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.436364,0.018182,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,55
HPSI0114i-lexy_2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.236364,0.000000,...,0.0,0.0,0.000000,0.054545,0.0,0.0,0.0,0.0,0.0,55
HPSI0114i-posc_1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.274510,0.000000,...,0.0,0.0,0.000000,0.098039,0.0,0.0,0.0,0.0,0.0,51


In [18]:
cell_count_ds = phenotype_df['n_cells']
cell_count_ds

HPSI0114i-bezi_3     42
HPSI0114i-eipl_1    129
HPSI0114i-kolf_2     55
HPSI0114i-lexy_2     55
HPSI0114i-posc_1     51
                   ... 
HPSI1113i-qorq_1    104
HPSI1113i-vuud_2     22
HPSI1114i-kuul_1    731
HPSI1114i-ualf_6     20
HPSI1213i-hehd_1    176
Name: n_cells, Length: 156, dtype: int64

In [19]:
noise_scaling_vector = [1/float(x) for x in cell_count_ds.tolist()]
noise_matrix = np.diag(noise_scaling_vector)

In [20]:
noise_matrix_df = pd.DataFrame(data=noise_matrix, index=cell_count_ds.index, columns=cell_count_ds.index)
noise_matrix_df.to_csv(noise_matrix_file, sep='\t')

In [21]:
cols_to_drop = list(set(phenotype_df.columns) & {'n_cells'})
phenotype_df = phenotype_df.drop(cols_to_drop, axis=1)
phenotype_df.head()

,MIR1302-10,FAM138A,OR4F5,RP11-34P13.7,RP11-34P13.8,AL627309.1,RP11-34P13.14,RP11-34P13.9,AP006222.2,RP4-669L17.10,...,KIR3DL2-1,AL590523.1,CT476828.1,PNRC2-1,SRSF10-1,AC145205.1,BAGE5,CU459201.1,AC002321.2,AC002321.1
HPSI0114i-bezi_3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.309524,0.000000,...,0.0,0.0,0.0,0.071429,0.142857,0.0,0.0,0.0,0.0,0.0
HPSI0114i-eipl_1,0.0,0.0,0.0,0.0,0.0,0.031008,0.0,0.0,0.472868,0.000000,...,0.0,0.0,0.0,0.000000,0.131783,0.0,0.0,0.0,0.0,0.0
HPSI0114i-kolf_2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.436364,0.018182,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
HPSI0114i-lexy_2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.236364,0.000000,...,0.0,0.0,0.0,0.000000,0.054545,0.0,0.0,0.0,0.0,0.0
HPSI0114i-posc_1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.274510,0.000000,...,0.0,0.0,0.0,0.000000,0.098039,0.0,0.0,0.0,0.0,0.0


In [22]:
phenotype_df = phenotype_df.transpose()
phenotype_df.head()

,HPSI0114i-bezi_3,HPSI0114i-eipl_1,HPSI0114i-kolf_2,HPSI0114i-lexy_2,HPSI0114i-posc_1,HPSI0114i-rozh_4,HPSI0114i-vass_1,HPSI0114i-wegi_1,HPSI0115i-aion_2,HPSI0115i-aoxv_1,...,HPSI1113i-bima_1,HPSI1113i-hayt_3,HPSI1113i-ieki_3,HPSI1113i-podx_1,HPSI1113i-qolg_1,HPSI1113i-qorq_1,HPSI1113i-vuud_2,HPSI1114i-kuul_1,HPSI1114i-ualf_6,HPSI1213i-hehd_1
MIR1302-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.013514,0.0,0.0,0.000000,0.0,0.0
FAM138A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
OR4F5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
RP11-34P13.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.001748,0.0,0.0,0.000000,0.0,0.0,0.009576,0.0,0.0
RP11-34P13.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.001368,0.0,0.0


In [23]:
gene_names = phenotype_df.index

In [24]:
#number of top expressed genes
selected_genes = list(phenotype_df.mean(axis=1).nlargest(n_genes).index)
phenotype_df = phenotype_df.loc[selected_genes, :]

In [25]:
phenotype_list = phenotype_df.index
mapping_df = mapping_df.query('hgnc_symbol in @phenotype_list')
mapping_df = mapping_df.drop_duplicates(subset=['hgnc_symbol'])
mapping_df = mapping_df.set_index('hgnc_symbol')

In [26]:
# limit only to hgnc symbols that map to ensembl gene IDs
phenotype_df = phenotype_df.loc[mapping_df.index,:]
phenotype_df.index = mapping_df['ensembl_gene_id']
phenotype_df.head()

,HPSI0114i-bezi_3,HPSI0114i-eipl_1,HPSI0114i-kolf_2,HPSI0114i-lexy_2,HPSI0114i-posc_1,HPSI0114i-rozh_4,HPSI0114i-vass_1,HPSI0114i-wegi_1,HPSI0115i-aion_2,HPSI0115i-aoxv_1,...,HPSI1113i-bima_1,HPSI1113i-hayt_3,HPSI1113i-ieki_3,HPSI1113i-podx_1,HPSI1113i-qolg_1,HPSI1113i-qorq_1,HPSI1113i-vuud_2,HPSI1114i-kuul_1,HPSI1114i-ualf_6,HPSI1213i-hehd_1
ensembl_gene_id,,,,,,,,,,,,,,,,,,,,,
ENSG00000261657,0.119048,0.348837,0.145455,0.181818,0.156863,0.416667,0.272727,0.163539,0.210256,0.302325,...,0.111111,0.138112,0.190476,0.193396,0.148649,0.375,0.090909,0.131327,0.05,0.164773
ENSG00000229483,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000
ENSG00000232849,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000
ENSG00000229558,0.000000,0.000000,0.000000,0.018182,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000
ENSG00000224394,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000


In [27]:
phenotype_df.to_csv(phenotype_file, sep='\t')

In [28]:
############ Covariates (15 PCs)

In [29]:
n_pcs = 15
pc_mat = sklearn.decomposition.PCA(n_components=n_pcs).fit_transform(phenotype_df.values.transpose())
pc_df = pd.DataFrame(data=pc_mat, index=phenotype_df.columns, columns=['PC{}'.format(x) for x in range(1,n_pcs+1)])
pc_df.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15
HPSI0114i-bezi_3,-2.945813,65.176968,-45.849983,26.253712,27.620760,37.661902,9.882468,-52.451080,0.136029,-25.960620,-7.205818,-15.165214,4.725231,12.651444,-2.881431
HPSI0114i-eipl_1,75.352142,58.404288,-16.074369,8.612268,11.714731,27.467228,-12.001464,1.453376,22.536844,4.726434,19.827853,-23.185532,14.766736,-6.194438,-4.251851
HPSI0114i-kolf_2,-92.370263,-6.198207,-22.377135,-14.397252,-9.570102,-32.989169,15.480437,-13.229457,1.235514,1.132164,-9.976591,-4.117060,3.172925,1.529822,-7.864626
HPSI0114i-lexy_2,62.473675,-30.429531,17.057215,-0.696849,27.544117,31.877274,-3.451139,-8.220388,0.158164,-4.434902,-1.385599,3.292422,7.805819,-2.689019,-9.816688
HPSI0114i-posc_1,-5.989481,-8.946047,7.160998,2.464357,38.258304,23.326028,6.391426,-28.516929,6.290118,-12.754497,-6.061685,11.927372,-5.233462,4.341463,-4.893327


In [30]:
pc_df.to_csv(covariates_file, sep='\t')